Vorwärtsauswertung

In [1]:
import tkinter as tk
from tkinter import ttk, messagebox
import tensorflow as tf
import numpy as np
from PIL import Image, ImageTk

# Modell wird beim Start der GUI geladen
try:
    model = tf.keras.models.load_model("Model/model.keras")
except Exception as e:
    print("Fehler beim Laden des Modells:", e)
    model = None


# Funktion zum Laden und Anzeigen eines Bildes auf dem Canvas
def load_image(canvas, image_path, x, y, size):
    try:
        # Bild wird geöffnet, auf die gewünschte Größe skaliert und auf dem Canvas angezeigt
        img = Image.open(image_path).resize(size)
        photo = ImageTk.PhotoImage(img)
        canvas.create_image(x, y, image=photo, anchor=tk.NW)
        # Referenz auf das Bild wird gespeichert, um das Bild im Speicher zu halten
        if not hasattr(canvas, "images"):
            canvas.images = []
        canvas.images.append(photo)
    except Exception as e:
        print(f"Error loading image {image_path}: {e}")

# Funktion zur Berechnung der Vorhersage
def berechne():
    if model is None:
        # Wenn kein Modell geladen ist, wird eine Fehlermeldung angezeigt
        for var in ausgabe_var:
            var.set("Kein Modell")
        messagebox.showerror("Fehler", "Kein Modell geladen. Bitte prüfen Sie den Pfad.")
        return

    try:
        # Eingabewerte werden normalisiert und in ein NumPy-Array umgewandelt
        divisoren = [810.10, 1.50, 15, 0.5, 75, 0.05]
        werte = [float(e.get()) / divisoren[i] for i, e in enumerate(eingabefelder)]
        eingabe_np = np.array(werte).reshape(1, -1)
        # Modellvorhersage wird berechnet
        prediction = model.predict(eingabe_np)
        # Ergebnisse werden in den Ausgabefeldern angezeigt
        for i, val in enumerate(prediction[0]):
            ausgabe_var[i].set(f"{val:.4f}")
    except ValueError:
        # Fehler bei ungültigen Eingabewerten
        for var in ausgabe_var:
            var.set("Ungültig")
        messagebox.showerror("Fehler", "Ungültige Eingabewerte. Bitte überprüfen Sie die Eingaben.")
    except Exception as e:
        # Allgemeiner Fehler bei der Berechnung
        for var in ausgabe_var:
            var.set("Fehler")
        print("Fehler bei der Berechnung:", e)
        messagebox.showerror("Fehler", "Unerwarteter Fehler bei der Berechnung.")

# GUI-Setup
root = tk.Tk()
root.title("Auslegung von Schnittflächen")
root.minsize(1050, 400)  # Mindestgröße des Fensters

# Haupt-Frame für die Eingabe- und Ausgabefelder
mainframe = ttk.Frame(root, padding="10")
mainframe.grid(row=0, column=1, sticky="NSEW")

# Canvas für die Bildanzeige links
canvas_left = tk.Canvas(root, width=282, height=268, bg="white")
canvas_left.grid(row=0, column=0, rowspan=8, padx=10, pady=5)

# Überschrift für die Eingabefelder
ttk.Label(mainframe, text="Scherschneidparamter", font=("Arial", 11, "bold")).grid(row=0, column=0, columnspan=2, pady=(0, 10), sticky="W")

# Labels und Slider für die Eingabewerte
eingabe_labels = [
    "Zugfestigkeit [MPa]",
    "Blechdicke [mm]",
    "Schneidspalt [%]",
    "Stegbreite [mm]",
    "Stegwinkel [°]",
    "Stempelradius [mm]"
]

slider_ranges = [
    (300, 800, 550, 1),  # Bereich, Standardwert und Schrittweite für jeden Slider
    (0.5, 1.5, 1, 0.1),
    (10, 15, 12.5, 0.1),
    (0.1, 0.5, 0.3, 0.025),
    (45, 75, 60, 1),
    (0.020, 0.050, 0.035, 0.005)
]

eingabefelder = []
for i, label in enumerate(eingabe_labels):
    # Label und Slider für jeden Eingabewert
    ttk.Label(mainframe, text=label).grid(row=i+1, column=0, sticky="W")
    slider = tk.Scale(mainframe, from_=slider_ranges[i][0], to=slider_ranges[i][1],
                      orient="horizontal", length=200, resolution=slider_ranges[i][3])
    slider.set(slider_ranges[i][2])
    slider.grid(row=i+1, column=1)
    eingabefelder.append(slider)

# Separator zwischen Eingabe- und Ausgabebereich
ttk.Separator(mainframe, orient='vertical').grid(row=0, column=2, rowspan=7, sticky='NS', padx=10)

# Überschrift für die Ausgabefelder
ttk.Label(mainframe, text="Schnittflächenkenngrößen", font=("Arial", 11, "bold")).grid(row=0, column=3, columnspan=2, pady=(0, 10), sticky="W")

# Labels und Felder für die Ausgabe
ausgabe_labels = [
    "Kanteneinzugsanteil KEA [%]",
    "Glattschnittanteil GSA [%]",
    "Bruchflächenanteil BFA [%]",
    "Gratanteil GRAT [%]"
]

ausgabefelder = []
ausgabe_var = []
for i, label in enumerate(ausgabe_labels):
    # Label und Ausgabe-Feld für jeden Wert
    ttk.Label(mainframe, text=label).grid(row=i+1, column=3, sticky="W")
    var = tk.StringVar()
    output = ttk.Label(mainframe, textvariable=var, width=20, relief="sunken")
    output.grid(row=i+1, column=4)
    ausgabefelder.append(output)
    ausgabe_var.append(var)

# Button zur Berechnung
button = ttk.Button(mainframe, text="Berechnung starten", command=berechne)
button.grid(row=7, column=0, columnspan=5, pady=10)

# Canvas für die Bildanzeige rechts
canvas_right = tk.Canvas(root, width=370, height=282, bg="white")
canvas_right.grid(row=0, column=3, rowspan=8, padx=10, pady=5)

# Bild auf dem linken Canvas laden
load_image(canvas_left,"GUI/HS.png", 0, 0, (281, 267))

# Bild auf dem rechten Canvas laden
load_image(canvas_right, "GUI/SK.png", 0, 0, (369, 281))

# Hauptloop der GUI
root.mainloop()


Rückwärtsauswertung

In [3]:
import tkinter as tk
from tkinter import ttk
from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf
from skopt import gp_minimize
import time

# Modell laden
try:
    model = tf.keras.models.load_model("Model/model.keras")  # Versuche, das Keras-Modell zu laden
except Exception as e:
    print(f"Fehler beim Laden des Modells: {e}")  # Fehlerbehandlung, falls das Modell nicht geladen werden kann
    exit(1)

# Normalisierungsfaktoren
normalization_factors = np.array([810, 1.5, 15, 0.5, 75, 0.05])  # Faktoren zur Normalisierung der Eingabewerte

# Fixe Eingaben (normalisiert)
fixed_inputs = {
    0: 600 / normalization_factors[0],  # Zugfestigkeit
    1: 1.0 / normalization_factors[1],  # Blechdicke
    2: 10 / normalization_factors[2],   # Schneidspalt
}

# Zielausgabe
target_index = 1  # Index der Zielausgabe (z. B. GSA)
y_target = 55  # Initialer Zielwert für den Glattschnittanteil (GSA)

# Optimierbare Indizes und Rohwerte
opt_indices = [3, 4, 5]  # Indizes der optimierbaren Parameter (Stegbreite, Stegwinkel, Stempelradius)
raw_bounds = [
    (0.1, 0.5),    # Stegbreite [mm]
    (45, 75),      # Stegwinkel [°]
    (0.020, 0.050) # Stempelradius [mm]
]
# Normalisierte Grenzen für die Optimierung
bounds = [(low / normalization_factors[idx], high / normalization_factors[idx])
          for (low, high), idx in zip(raw_bounds, opt_indices)]

# Bild laden und anzeigen
def load_image(canvas, image_path, x, y, size):
    try:
        img = Image.open(image_path).resize(size)  # Bild öffnen und skalieren
        photo = ImageTk.PhotoImage(img)  # Bild in ein tkinter-kompatibles Format konvertieren
        canvas.create_image(x, y, image=photo, anchor=tk.NW)  # Bild auf dem Canvas platzieren
        if not hasattr(canvas, "images"):
            canvas.images = []  # Liste zur Vermeidung von Garbage Collection
        canvas.images.append(photo)
    except Exception as e:
        print(f"Fehler beim Laden des Bildes {image_path}: {e}")  # Fehlerbehandlung

# Zielfunktion für die Optimierung
def objective(opt_inputs):
    full_input = np.zeros(6)  # Initialisiere Eingabevektor
    for i, idx in enumerate(opt_indices):
        full_input[idx] = opt_inputs[i]  # Setze optimierbare Eingaben
    for idx, val in fixed_inputs.items():
        full_input[idx] = val  # Setze fixe Eingaben
    y_pred = model.predict(full_input.reshape(1, -1), verbose=0)  # Vorhersage mit dem Modell
    return (y_pred[0, target_index] - y_target) ** 2  # Fehlerquadratsumme als Zielfunktion

# Optimierung
def optimize_inputs(objective, bounds, fixed_inputs, opt_indices, normalization_factors):
    x0 = [np.random.uniform(low, high) for low, high in bounds]  # Initiale Werte für die Optimierung
    result = gp_minimize(objective, bounds, n_calls=50, random_state=42, n_initial_points=10, acq_func="EI")  # Optimierung mit Gaussian Process
    x_optim_partial = result.x  # Optimierte Werte
    x_full_normed = np.zeros(len(normalization_factors))  # Normalisierter Eingabevektor
    for i, idx in enumerate(opt_indices):
        x_full_normed[idx] = x_optim_partial[i]  # Setze optimierte Werte
    for idx, val in fixed_inputs.items():
        x_full_normed[idx] = val  # Setze fixe Werte
    x_full_raw = x_full_normed * normalization_factors  # Denormalisiere die Werte
    return x_full_raw, x_full_normed

# Aktion beim Klick auf "Berechnung starten"
def on_calculate():
    # Werte aus den ersten drei Slidern extrahieren (Zugfestigkeit, Blechdicke, Schneidspalt)
    fixed_inputs[0] = eingabefelder[0].get() / normalization_factors[0]  # Zugfestigkeit
    fixed_inputs[1] = eingabefelder[1].get() / normalization_factors[1]  # Blechdicke
    fixed_inputs[2] = eingabefelder[2].get() / normalization_factors[2]  # Schneidspalt
    
    # Zielwert (Glattschnittanteil) aus dem vierten Slider holen
    global y_target
    y_target = eingabefelder[3].get()  # Glattschnittanteil [%]

    start_time = time.time()  # Startzeit für die Optimierung

    # Optimierungsfunktion aufrufen
    x_full_raw, x_full_normed = optimize_inputs(objective, bounds, fixed_inputs, opt_indices, normalization_factors)

    # Vorhersagen vom Modell holen
    y_final = model.predict(x_full_normed.reshape(1, -1), verbose=0)

    # Aktualisiere die Ausgabefelder mit den vorhergesagten Werten
    for i, value in enumerate(y_final[0]):
        ausgabe_var1[i].set(f"{value:.2f}")
    for i, value in enumerate(x_full_raw[-3:]):
        ausgabe_var2[i].set(f"{value:.2f}")

    # Berechnungsdauer ausgeben
    duration = time.time() - start_time
    print(f"Optimierung abgeschlossen in {duration:.2f} Sekunden.")
    print("Optimierte Eingaben (Rohwerte):", x_full_raw)
    print("Vorhergesagte Ausgaben:", y_final[0])

# GUI
root = tk.Tk()
root.title("Auslegung von Schnittflächen")  # Titel des Fensters

# Haupt-Frame mit Padding
mainframe = ttk.Frame(root, padding="10")
mainframe.grid(row=0, column=1, sticky="NSEW")

# Canvas für Bildanzeige rechts
canvas_right = tk.Canvas(root, width=370, height=282, bg="white")
canvas_right.grid(row=0, column=3, rowspan=8, padx=10, pady=5)
# Bild auf dem rechten Canvas laden
load_image(canvas_right, "GUI/SK.png", 0, 0, (369, 281))

# Canvas für Bildanzeige links (neben den Slidern)
canvas_left = tk.Canvas(root, width=282, height=268, bg="white")
canvas_left.grid(row=0, column=0, rowspan=8, padx=10, pady=5)
# Bild auf dem linken Canvas laden
load_image(canvas_left, "GUI/HS.png", 0, 0, (281, 267))

# Eingabe-Labels und Slider
eingabe_labels = [
    "Zugfestigkeit WS [MPa]",
    "Blechdicke s [mm]",
    "Schneidspalt u [%]",
    "Glattschnittanteil GSA [%]"
]
slider_ranges = [
    (300, 800, 550, 1),  # Bereich und Standardwert für Zugfestigkeit
    (0.5, 1.5, 1, 0.1),  # Bereich und Standardwert für Blechdicke
    (10, 15, 12.5, 0.1), # Bereich und Standardwert für Schneidspalt
    (25, 80, 50, 1),     # Bereich und Standardwert für Glattschnittanteil
]

# Funktion zum Erstellen eines Sliders
def create_slider(label, range_values, row):
    ttk.Label(mainframe, text=label).grid(row=row, column=0, sticky="W")  # Label für den Slider
    slider = tk.Scale(mainframe, from_=range_values[0], to=range_values[1],
                      orient="horizontal", length=200, resolution=range_values[3])  # Slider erstellen
    slider.set(range_values[2])  # Standardwert setzen
    slider.grid(row=row, column=1)  # Slider positionieren
    return slider

# Slider für die Eingabewerte erstellen
eingabefelder = [create_slider(label, slider_ranges[i], i+1) for i, label in enumerate(eingabe_labels)]

# Überschriften über den Ausgabeblöcken
ttk.Label(mainframe, text="Eingangsparameter", font=("Arial", 10, "bold")).grid(row=0, column=0, columnspan=2, pady=(0, 5))
ttk.Label(mainframe, text="Schnittflächenkenngrößen", font=("Arial", 10, "bold")).grid(row=0, column=2, columnspan=2, pady=(0, 5))
ttk.Label(mainframe, text="Optimierte Schneidparameter", font=("Arial", 10, "bold")).grid(row=0, column=4, columnspan=2, pady=(0, 5))

# Modellausgaben (KEA, GSA, BFA, GRAT)
ausgabe_labels = [
    "Kanteneinzugsanteil KEA [%]",
    "Glattschnittanteil GSA [%]",
    "Bruchflächenanteil BFA [%]",
    "Gratanteil GRAT [%]"
]

ausgabefelder1 = []
ausgabe_var1 = []
for i, label in enumerate(ausgabe_labels):
    ttk.Label(mainframe, text=label).grid(row=i+1, column=2, sticky="W")  # Label für die Ausgabe
    var = tk.StringVar()  # Variable für die Ausgabe
    output = ttk.Label(mainframe, textvariable=var, width=20, relief="sunken")  # Ausgabe-Label
    output.grid(row=i+1, column=3)  # Positionieren
    ausgabefelder1.append(output)
    ausgabe_var1.append(var)

# Optimierte Eingaben (Stegbreite, Stegwinkel, Stempelradius)
ausgabe_labels2 = [
    "Stegbreite SB [mm]",
    "Stegwinkel SW [°]",
    "Stempelradius SR [mm]"
]
ausgabefelder2 = []
ausgabe_var2 = []
for i, label in enumerate(ausgabe_labels2):
    ttk.Label(mainframe, text=label).grid(row=i+1, column=4, sticky="W")  # Label für die optimierten Eingaben
    var = tk.StringVar()  # Variable für die Ausgabe
    output = ttk.Label(mainframe, textvariable=var, width=20, relief="sunken")  # Ausgabe-Label
    output.grid(row=i+1, column=5)  # Positionieren
    ausgabefelder2.append(output)
    ausgabe_var2.append(var)

# Button für Berechnung
button = ttk.Button(mainframe, text="Berechnung starten", command=on_calculate)
button.grid(row=8, column=1, columnspan=4, pady=10)

# Start GUI-Loop
root.mainloop()  # GUI starten
